<a href="https://colab.research.google.com/github/RiturajSaha/App-Review-Analyzer/blob/main/NLPmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/Review Analysis'

/content/drive/MyDrive/Colab Notebooks/Review Analysis


Importing the Libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import pickle
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Defining required Functions

In [ ]:
def classifyCol(df,col):
  from sklearn.preprocessing import LabelEncoder
  le = LabelEncoder()
  df[col] = le.fit_transform(df[col])
  print('For',col,'column:')
  print(dict(zip(le.classes_, le.transform(le.classes_))))
  return df


def cleanCol(df,col):
  reviews=[]
  for row in df[col]:
     row=re.sub('[^a-z]', ' ', row.lower())
     stopWords = nltk.corpus.stopwords.words('english')
     stopWords.remove('not')
     row=row.split()
     row = [nltk.stem.porter.PorterStemmer().stem(word) for word in row if not word in set(stopWords)]
     row = ' '.join(row)
     reviews.append(row)
  df[col]=reviews
  return df

def getAcc(model, x_train, x_test, y_train, y_test) : 
  from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, confusion_matrix, classification_report 
  y_pred = model.predict(x_test)
  print("Accuracy:",accuracy_score(y_test, y_pred)*100) 
  print("R2 Score:",r2_score(y_test, y_pred)) 
  print("Mean Absolute Error:",mean_absolute_error(y_test, y_pred))
  print("Confusion Matrix:\n",confusion_matrix(y_test, y_pred))
  print("Classification Report:\n",classification_report(y_test, y_pred)) 

def bagOfWords(x):
  from sklearn.feature_extraction.text import CountVectorizer
  cv = CountVectorizer()
  vector = cv.fit_transform(x).toarray()
  mf=len(vector[0])
  mf=int(str(mf)[:-2]+'00')
  cv = CountVectorizer(max_features=mf)
  x = cv.fit_transform(x).toarray()
  return (x)
   

Importing the Dataset

In [ ]:
df=pd.read_csv('googleplaystore_user_reviews.csv')
display(df)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
...,...,...,...,...,...
64290,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN
64291,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN
64292,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN
64293,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN


Cleaning and Processing the Dataframe

In [ ]:
df.dropna(inplace=True)
df=df.iloc[:,1:3]
df.rename(columns = {'Translated_Review':'REVIEW', 
                       'Sentiment':'SENTIMENT'}, inplace = True)
df=classifyCol(df,'SENTIMENT')
df=cleanCol(df,'REVIEW')
display(df)

For SENTIMENT column:
{'Negative': 0, 'Neutral': 1, 'Positive': 2}


,REVIEW,SENTIMENT
0,like eat delici food cook food case best food ...,2
1,help eat healthi exercis regular basi,2
3,work great especi go groceri store,2
4,best idea us,2
5,best way,2
...,...,...
64222,ad older mani agent not much owner post detail...,2
64223,photo post portal load fit purpos sure storag ...,2
64226,dumb app want post properti rent give option w...,0
64227,properti busi got link sm happi perform still ...,2


Applying Bag Of Words model to the REVIEW column

In [ ]:
x=bagOfWords(df['REVIEW'])
y=df['SENTIMENT']

Splitting the Dataframe into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

print("x_train:",len(x_train))
print("x_test:",len(x_test))
print("y_train:",len(y_train))
print("y_test:",len(y_test))

x_train: 29941
x_test: 7486
y_train: 29941
y_test: 7486


Applying Random Forest Classification model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC_model = RandomForestClassifier(n_estimators = 75, criterion = 'entropy', random_state = 0)
RFC_model.fit(x_train, y_train)
getAcc(RFC_model, x_train, x_test, y_train, y_test)

Accuracy: 90.11488111140797
R2 Score: 0.6030273213438854
Mean Absolute Error: 0.15749398877905424
Confusion Matrix:
 [[1235   58  394]
 [  16  900  144]
 [  45   83 4611]]
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.73      0.83      1687
           1       0.86      0.85      0.86      1060
           2       0.90      0.97      0.93      4739

    accuracy                           0.90      7486
   macro avg       0.90      0.85      0.87      7486
weighted avg       0.90      0.90      0.90      7486



Applying Linear Support Vector Classifcation model

In [ ]:
from sklearn.svm import LinearSVC
LSVC_model = LinearSVC()
LSVC_model.fit(x_train, y_train)
getAcc(LSVC_model, x_train, x_test, y_train, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Accuracy: 91.11675126903553
R2 Score: 0.6742392408500139
Mean Absolute Error: 0.13438418380977826
Confusion Matrix:
 [[1445   55  187]
 [  48  903  109]
 [ 154  112 4473]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.86      0.87      1687
           1       0.84      0.85      0.85      1060
           2       0.94      0.94      0.94      4739

    accuracy                           0.91      7486
   macro avg       0.89      0.88      0.89      7486
weighted avg       0.91      0.91      0.91      7486



Applying Logistic Regression model

In [ ]:
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(random_state = 0)
LR_model.fit(x_train, y_train)
getAcc(LR_model, x_train, x_test, y_train, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy: 91.45070798824473
R2 Score: 0.6796428553382838
Mean Absolute Error: 0.13091103393000267
Confusion Matrix:
 [[1433   64  190]
 [  26  938   96]
 [ 150  114 4475]]
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.85      0.87      1687
           1       0.84      0.88      0.86      1060
           2       0.94      0.94      0.94      4739

    accuracy                           0.91      7486
   macro avg       0.89      0.89      0.89      7486
weighted avg       0.91      0.91      0.91      7486



Saving the Model having highest Accuracy

In [ ]:
#pickle.dump(LR_model, open('LR_model.pkl','wb')) 
model = pickle.load(open('LR_model.pkl','rb'))

Preparing dataframe to apply the saved Model

In [ ]:
df=pd.read_csv('googleplaystore_user_reviews.csv')
display(df)

df.dropna(inplace=True)
df=df.iloc[:,1:3]
df.rename(columns = {'Translated_Review':'REVIEW', 
                       'Sentiment':'SENTIMENT'}, inplace = True)
df=classifyCol(df,'SENTIMENT')
df=cleanCol(df,'REVIEW')
display(df)

x=bagOfWords(df['REVIEW'])
y=df['SENTIMENT']


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
...,...,...,...,...,...
64290,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN
64291,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN
64292,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN
64293,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN


For SENTIMENT column:
{'Negative': 0, 'Neutral': 1, 'Positive': 2}


,REVIEW,SENTIMENT
0,like eat delici food cook food case best food ...,2
1,help eat healthi exercis regular basi,2
3,work great especi go groceri store,2
4,best idea us,2
5,best way,2
...,...,...
64222,ad older mani agent not much owner post detail...,2
64223,photo post portal load fit purpos sure storag ...,2
64226,dumb app want post properti rent give option w...,0
64227,properti busi got link sm happi perform still ...,2


Comparing the Original output with the Predicted output

In [ ]:
y_pred=model.predict(x)

df['PREDICTED SENTIMENT']=y_pred
df.rename(columns = {'SENTIMENT':'ORIGINAL SENTIMENT'}, inplace = True)
df['ORIGINAL SENTIMENT'].replace({0: 'Negative', 
                                  1: 'Neutral', 
                                  2: 'Positive'}, inplace=True)
df['PREDICTED SENTIMENT'].replace({0: 'Negative', 
                                   1: 'Neutral', 
                                   2: 'Positive'}, inplace=True)

display(df.sample(10))

,REVIEW,ORIGINAL SENTIMENT,PREDICTED SENTIMENT
57807,grubhub almost everi day driver custom top notch,Positive,Positive
12317,love pet inform onr place easi access even rec...,Positive,Positive
35813,nice,Positive,Positive
44960,fake,Negative,Negative
20249,horribl app approach date random profil day ze...,Negative,Negative
51847,english pleas pleas provid option chang langua...,Positive,Positive
32503,love far fan age say fast pace keep get distract,Positive,Positive
16554,koskoca netflix give free trial period month y...,Positive,Positive
3695,transact attach facil go may upload doc take t...,Neutral,Neutral
49636,like everi say stream anyth video blank live s...,Negative,Positive
